# Chapter 5 - Ensemble machine learning, deep learning

2022 February 23

![kandc](img/kandc.jpg)

[Texas Monthly, Music Monday: Uncovering The Mystery Of The King & Carter Jazzing Orchestra](https://www.texasmonthly.com/the-daily-post/music-monday-uncovering-the-mystery-of-the-king-carter-jazzing-orchestra/)

## Ensemble machine learning

"Ensemble machine learning methods use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms." [H2O.ai ensemble example](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html)

In this manner, SuperLearner ensembles are powerful tools because they: 
* elucidate issues of algorithmic bias and variance
* circumvent bias introduced by selecting single models
* offer a means to optimize prediction through the stacking/blending of weaker models
* allow for comparison of multiple algorithms, and/or comparison of the same model but tuned in many different ways
* utilize a second-level algorithm that produces an ideal weighted prediction that is suitable for data of virtually all distributions and uses cross-validation to prevent overfitting

The below example utilizes the h2o package, and requires Java to be installed on your machine.
* install Java: https://www.java.com/en/download/help/mac_install.html
* h2o SuperLearner example: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html

Check out some other great tutorials: 
* Python mlens library: https://mlens.readthedocs.io/en/0.1.x/install/
* Machine Learning Mastery: https://machinelearningmastery.com/super-learner-ensemble-in-python/
* KDNuggets: https://www.kdnuggets.com/2018/02/introduction-python-ensembles.html/2#comments

The quintessential R guide: 
* Guide to SuperLearner: https://cran.r-project.org/web/packages/SuperLearner/vignettes/Guide-to-SuperLearner.html

Read the papers: 
* [Van der Laan, M.J.; Polley, E.C.; Hubbard, A.E. Super Learner. Stat. Appl. Genet. Mol. Biol. 2007, 6, 1–21.](https://www.degruyter.com/document/doi/10.2202/1544-6115.1309/html)
* [Polley, E.C.; van der Laan, M.J. Super Learner in Prediction, UC Berkeley Division of Biostatistics Working Paper Series Paper 266.](https://biostats.bepress.com/ucbbiostat/paper266)

## H2O SuperLearner ensemble

In [49]:
# !pip install h2o

# Requires install of Java
# https://www.java.com/en/download/help/mac_install.html

In [21]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 mins 11 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,5 months and 8 days !!!
H2O_cluster_name:,H2O_from_python_evanmuzzall_gwre63
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.549 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [22]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
train

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
1,0.869293,-0.635082,0.22569,0.32747,-0.689993,0.754202,-0.248573,-1.09206,0,1.37499,-0.653674,0.930349,1.10744,1.1389,-1.5782,-1.04699,0,0.65793,-0.0104546,-0.0457672,3.10196,1.35376,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.907542,0.329147,0.359412,1.49797,-0.31301,1.09553,-0.557525,-1.58823,2.17308,0.812581,-0.213642,1.27101,2.21487,0.499994,-1.26143,0.732156,0,0.398701,-1.13893,-0.00081911,0,0.30222,0.833048,0.9857,0.978098,0.779732,0.992356,0.798343
1,0.798835,1.47064,-1.63597,0.453773,0.425629,1.10487,1.28232,1.38166,0,0.851737,1.54066,-0.81969,2.21487,0.99349,0.35608,-0.208778,2.54822,1.25695,1.12885,0.900461,0,0.909753,1.10833,0.985692,0.951331,0.803252,0.865924,0.780118
0,1.34438,-0.876626,0.935913,1.99205,0.882454,1.78607,-1.64678,-0.942383,0,2.42326,-0.676016,0.736159,2.21487,1.29872,-1.43074,-0.364658,0,0.745313,-0.678379,-1.36036,0,0.946652,1.0287,0.998656,0.728281,0.8692,1.02674,0.957904
1,1.10501,0.321356,1.5224,0.882808,-1.20535,0.681466,-1.07046,-0.921871,0,0.800872,1.02097,0.971407,2.21487,0.596761,-0.350273,0.631194,0,0.479999,-0.373566,0.113041,0,0.755856,1.36106,0.98661,0.838085,1.1333,0.872245,0.808487
0,1.59584,-0.607811,0.00707492,1.81845,-0.111906,0.84755,-0.566437,1.58124,2.17308,0.755421,0.64311,1.42637,0,0.921661,-1.19043,-1.61559,0,0.651114,-0.654227,-1.27434,3.10196,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
1,0.409391,-1.88468,-1.02729,1.67245,-1.6046,1.33801,0.0554274,0.0134659,2.17308,0.509783,-1.03834,0.707862,0,0.746918,-0.358465,-1.64665,0,0.367058,0.0694965,1.37713,3.10196,0.869418,1.22208,1.00063,0.545045,0.698653,0.977314,0.828786
1,0.933895,0.62913,0.527535,0.238033,-0.966569,0.547811,-0.0594392,-1.70687,2.17308,0.941003,-2.65373,-0.15722,0,1.03037,-0.175505,0.523021,2.54822,1.37355,1.29125,-1.46745,0,0.901837,1.08367,0.979696,0.7833,0.849195,0.894356,0.774879
1,1.40514,0.536603,0.689554,1.17957,-0.110061,3.2024,-1.52696,-1.57603,0,2.93154,0.567342,-0.130033,2.21487,1.78712,0.899499,0.585151,2.54822,0.401865,-0.151202,1.16349,0,1.66707,4.03927,1.17583,1.04535,1.54297,3.53483,2.74075
1,1.17657,0.104161,1.397,0.479721,0.265513,1.13556,1.53483,-0.253291,0,1.02725,0.534316,1.18002,0,2.40566,0.0875568,-0.976534,2.54822,1.25038,0.268541,0.530334,0,0.833175,0.773968,0.98575,1.1037,0.84914,0.937104,0.812364


In [23]:
print(train.shape)
print(test.shape)

(10000, 29)
(5000, 29)


In [24]:
# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

In [25]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [26]:
# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

In [27]:
# There are a few ways to assemble a list of models to stack together:
# 1. Train individual models and put them in a list
# 2. Train a grid of models
# 3. Train several grids of models
# Note: All base models must have the same cross-validation folds and
# the cross-validated predicted values must be kept.


# 1. Generate a 2-model ensemble (GBM + RF)

# Train and cross-validate a GBM
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)
my_gbm.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1645639753735_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,10.0,10.0,1580.0,3.0,3.0,3.0,8.0,8.0,8.0




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.20052170746266884
RMSE: 0.44779650228945383
LogLoss: 0.5879177464092424
Mean Per-Class Error: 0.29613223631461116
AUC: 0.7735466157694937
AUCPR: 0.7909110775032231
Gini: 0.5470932315389874

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4424199442800167: 


,,0,1,Error,Rate
0,0,2433.0,2272.0,0.4829,(2272.0/4705.0)
1,1,818.0,4477.0,0.1545,(818.0/5295.0)
2,Total,3251.0,6749.0,0.309,(3090.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-02-23 10:12:55,1.846 sec,0.0,0.499129,0.691406,0.500000,0.529500,1.000000,0.4705
1,,2022-02-23 10:12:55,1.925 sec,1.0,0.488236,0.669779,0.689191,0.674970,1.340835,0.3927
2,,2022-02-23 10:12:55,1.945 sec,2.0,0.479807,0.653146,0.719111,0.725388,1.545030,0.3958
3,,2022-02-23 10:12:55,1.962 sec,3.0,0.473698,0.641037,0.724650,0.731284,1.569032,0.3933
4,,2022-02-23 10:12:55,1.984 sec,4.0,0.467046,0.627816,0.746638,0.760063,1.683844,0.3692
5,,2022-02-23 10:12:55,2.003 sec,5.0,0.462994,0.619629,0.750899,0.766012,1.696473,0.3404
6,,2022-02-23 10:12:55,2.022 sec,6.0,0.458301,0.609973,0.756603,0.772386,1.733566,0.3515
7,,2022-02-23 10:12:55,2.043 sec,7.0,0.455569,0.604192,0.758101,0.773795,1.748075,0.3512
8,,2022-02-23 10:12:55,2.067 sec,8.0,0.452777,0.598335,0.763715,0.778986,1.754413,0.3180
9,,2022-02-23 10:12:55,2.091 sec,9.0,0.449743,0.591929,0.768491,0.784821,1.776659,0.3207



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x26,565.332336,1.000000,0.410859
1,x28,204.503998,0.361741,0.148625
2,x27,189.230789,0.334725,0.137525
3,x23,124.527618,0.220273,0.090501
4,x6,121.551117,0.215008,0.088338
5,x25,99.598267,0.176176,0.072384
6,x4,44.359753,0.078467,0.032239
7,x10,18.008587,0.031855,0.013088
8,x22,3.660778,0.006475,0.002660
9,x18,3.097153,0.005478,0.002251



See the whole table with table.as_data_frame()


In [28]:
# Train and cross-validate a RF
my_rf = H2ORandomForestEstimator(ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1)
my_rf.train(x=x, y=y, training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1645639753735_129


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,943584.0,20.0,20.0,20.0,1402.0,1583.0,1496.78




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.20546981364562208
RMSE: 0.45328778236967965
LogLoss: 0.6008794722074892
Mean Per-Class Error: 0.3224386087972231
AUC: 0.7416906451357175
AUCPR: 0.7551298236228338
Gini: 0.48338129027143495

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3888572089486607: 


,,0,1,Error,Rate
0,0,1938.0,2767.0,0.5881,(2767.0/4705.0)
1,1,662.0,4633.0,0.125,(662.0/5295.0)
2,Total,2600.0,7400.0,0.3429,(3429.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-02-23 10:13:13,10.307 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-02-23 10:13:13,10.337 sec,1.0,0.642818,14.048395,0.577384,0.589593,1.144218,0.464917
2,,2022-02-23 10:13:13,10.369 sec,2.0,0.615404,11.895281,0.595767,0.598674,1.167648,0.470126
3,,2022-02-23 10:13:13,10.403 sec,3.0,0.596109,10.322973,0.606984,0.606263,1.188141,0.471944
4,,2022-02-23 10:13:13,10.435 sec,4.0,0.578952,8.963477,0.617051,0.613281,1.205225,0.473260
5,,2022-02-23 10:13:13,10.468 sec,5.0,0.566757,7.823122,0.621447,0.619131,1.222342,0.472650
6,,2022-02-23 10:13:13,10.500 sec,6.0,0.555494,6.782681,0.625539,0.623303,1.235544,0.472576
7,,2022-02-23 10:13:13,10.531 sec,7.0,0.542668,5.762143,0.634190,0.632386,1.256193,0.470778
8,,2022-02-23 10:13:13,10.565 sec,8.0,0.534223,5.001247,0.637938,0.634830,1.257749,0.470999
9,,2022-02-23 10:13:13,10.599 sec,9.0,0.525930,4.327270,0.643458,0.638581,1.259426,0.416980



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x26,7803.849121,1.000000,0.101371
1,x27,4741.325195,0.607562,0.061589
2,x28,4625.907227,0.592773,0.060090
3,x23,3613.875244,0.463089,0.046944
4,x25,3548.924072,0.454766,0.046100
5,x6,3477.234863,0.445579,0.045169
6,x4,3103.802002,0.397727,0.040318
7,x1,2902.952637,0.371990,0.037709
8,x10,2854.255615,0.365750,0.037077
9,x7,2711.995605,0.347520,0.035229



See the whole table with table.as_data_frame()


In [29]:
# Train a stacked ensemble using the GBM and GLM above
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm, my_rf])
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


In [30]:
# Compare to base learner performance on the test set
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

Best Base-learner Test AUC:  0.769204725074508
Ensemble Test AUC:  0.7731183158978566


In [31]:
# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)


# 2. Generate a random grid of models and stack them together

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [32]:
# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=nfolds,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=x, y=y, training_frame=train)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
    col_sample_rate learn_rate max_depth sample_rate  \
0               0.4       0.03         3         0.7   
1               0.2       0.03         4         0.8   
2               0.7       0.01         5         0.9   

                   model_ids             logloss  
0  gbm_grid_binomial_model_3   0.667209869083727  
1  gbm_grid_binomial_model_2  0.6742791229554091  
2  gbm_grid_binomial_model_1  0.6770690446978346  


In [33]:
# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%
Best Base-learner Test AUC:  0.748146530400473
Ensemble Test AUC:  0.7510921003414699
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


## Deep learning basics

Deep learning is a subfield of machine learning that uses a variety of multi-layered artificial neural networks to model datasets and predict outcomes. Deep learning is ideal for numeric, text, image, video, and sound data because deep representative networks store these data as large matrices and recycle error to make better predictions during the next epoch. 

To understand deep networks, let's start with a toy example of a single feed forward neural network - a perceptron.

Read Goodfellow et al's Deep Learning Book to learn more: https://www.deeplearningbook.org/

In [46]:
# generate toy dataset
example = {'x1': [1, 0, 1, 1, 0], 
           'x2': [1, 1, 1, 1, 0], 
           'xm': [1, 0, 1, 1, 0],
           'y': ['yes', 'no', 'yes', 'yes', 'no']
           }
example_df = pd.DataFrame(data = example)
example_df

,x1,x2,xm,y
0,1,1,1,yes
1,0,1,0,no
2,1,1,1,yes
3,1,1,1,yes
4,0,0,0,no


![perceptron](img/perceptron.png)

Perceptron figure modified from [Sebastian Raschka's Single-Layer Neural Networks and Gradient Descent](https://sebastianraschka.com/Articles/2015_singlelayer_neurons.html)

Perceptron key terms: 
* **Layer:** the network typology of a deep learning model, usually divided into variations of input, hidden, preprocessing, encoder/decoder, and output. 
* **Inputs:** the features/covariates for a single observation. These are just the individual cells in a dataframe (the 1's and 0s from `example_df` above), but they could be words from a text or pixels from an image. 
* **Weights:** the learnable parameters of a model that connect the input layer to the output via the net input (summation) and activation functions. 
* **Bias term:** A placeholder "1" assures that we do not receive 0 outputs by default. 
* **Net input function:** computes the weighted sum of the input layer. 
* **Activation function:** determine if a neuron should be fired or not. In binary classification for example, this means should a 1 or 0 be output?
* Output: one node that contains the y prediction
* **Error:** how far off an output prediction was. The weights can be updated by adjusting the learning rate based on the error to reduce it for the next epoch

## What makes a network "deep"?

A "deep" network is just network with multiple/many hidden layers for handling potential nonlinear transformations.

* Fully connected layer: a layer where all nodes are connected to every node in the next layer (as indicated by the purple arrows 

![deep](img/deep.png)

Example of "deep" network with two hidden layers modified from [DevSkrol's Artificial Neural Network Explained with an Regression Example](https://devskrol.com/2020/11/22/388/)

>NOTE: Bias term not shown for some reason!

Let's go through François Chollet's "Image classification from scratch" [tutorial](https://keras.io/examples/vision/image_classification_from_scratch/) to examine this architecture to predict images of cats versus dogs. 

[Click here to open the Colab notebook](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/image_classification_from_scratch.ipynb)

You should also check out his deep learning book! https://www.manning.com/books/deep-learning-with-python-second-edition

![dogcat](img/dogcat.jpg)